In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000
! mkdir data
! unzip -q skin-cancer-mnist-ham10000.zip -d data

Dataset URL: https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000
License(s): CC-BY-NC-SA-4.0
100% 5.20G/5.20G [04:10<00:00, 23.5MB/s]
100% 5.20G/5.20G [04:10<00:00, 22.3MB/s]


In [1]:
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification, ViTFeatureExtractor
from torchvision import transforms
from torch import nn, optim
from collections import defaultdict
from sklearn.metrics import accuracy_score
from PIL import Image
import itertools
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from itertools import product
from tqdm import tqdm  # For progress bar

# Constants
DATA_DIR = 'data/'
METADATA_FILE = os.path.join(DATA_DIR, 'HAM10000_metadata.csv')
IMAGE_DIRS = [os.path.join(DATA_DIR, 'HAM10000_images_part_1'),
              os.path.join(DATA_DIR, 'HAM10000_images_part_2')]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Configurable hyperparameters
EPOCHS = 10   # Set the number of epochs here
BATCH_SIZE = 256   # Set the batch size here
PATIENCE = 5  # Early stopping patience (number of epochs with no improvement)

# Load Metadata
metadata = pd.read_csv(METADATA_FILE)

# Encode the string labels into numeric values
label_encoder = LabelEncoder()
metadata['dx_encoded'] = label_encoder.fit_transform(metadata['dx'])

# Initialize Feature Extractor
MODEL_NAME = 'google/vit-base-patch32-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(MODEL_NAME)

# Augmentation Transforms
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
])

# Custom Dataset (Using Encoded Labels)
class SkinCancerDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, image_dirs, feature_extractor, transform=None):
        self.dataframe = dataframe
        self.image_dirs = image_dirs
        self.feature_extractor = feature_extractor
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_id = row['image_id']
        # Search for the image in the provided directories
        for directory in self.image_dirs:
            image_path = os.path.join(directory, f"{image_id}.jpg")
            if os.path.exists(image_path):
                break
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        # Using 'dx_encoded' as the label column
        label = torch.tensor(row['dx_encoded'])  # Use the encoded labels
        return image, label

# Your provided stratifiedKFold class
class stratifiedKFold:
    def __init__(self, n_split, shuffle, random_state):
        self.n_split = n_split
        self.shuffle = shuffle
        self.random_state = random_state

    def split(self, X, y):
        if (self.shuffle and self.random_state) is not None:
            np.random.seed(self.random_state)

        idx = np.arange(len(y))
        if self.shuffle:
            np.random.shuffle(idx)

        y_idx = defaultdict(list)
        for i, label in zip(idx, y):
            y_idx[label].append(i)

        splits = [[] for _ in range(self.n_split)]

        for label, i in y_idx.items():
            np.random.shuffle(i)
            split_portions = [len(i) // self.n_split] * self.n_split
            for j in range(len(i) % self.n_split):
                split_portions[j] += 1

            first = 0
            for k in range(self.n_split):
                last = first + split_portions[k]
                splits[k].extend(i[first:last])
                first = last

        for i in range(self.n_split):
            test_idx = splits[i]
            train_idx = np.concatenate([splits[j] for j in range(self.n_split) if j != i])
            yield train_idx, test_idx

    def get_n_splits(self):
        return self.n_split

# Stratified sampling to get 30% of the data
def stratified_sample(dataframe, labels, sample_size=0.3, random_state=42):
    n_samples = int(sample_size * len(dataframe))
    skf = stratifiedKFold(n_split=int(1 / sample_size), shuffle=True, random_state=random_state)

    # We only need the first split for 30% sampling
    for train_idx, test_idx in skf.split(dataframe, labels):
        return dataframe.iloc[test_idx[:n_samples]]

# Sample 30% of the data
metadata_sample = stratified_sample(metadata, metadata['dx_encoded'], sample_size=0.3)

# Reset index for the sampled data
metadata_sample = metadata_sample.reset_index(drop=True)

# Define hyperparameters to test (learning rate and attention heads)
hyperparameters = [
    {'lr': [1e-3, 1e-4]},
    {'attention_heads': [8, 16]}  # Example values
]

# Training and evaluation loops with progress bar
def train_epoch(model, train_loader, optimizer, criterion, device, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Create a progress bar for batch processing
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1} Training", leave=False)

    for batch_idx, (images, labels) in enumerate(progress_bar):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).logits  # logits from ViTForImageClassification
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Update progress bar with batch loss and accuracy
        progress_bar.set_postfix({'Batch Loss': loss.item(), 'Batch Acc': 100 * correct / total})

    # Return average loss and accuracy for the epoch
    return running_loss / total, correct / total

def eval_epoch(model, val_loader, criterion, device, epoch):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    # Create a progress bar for batch processing during evaluation
    progress_bar = tqdm(val_loader, desc=f"Epoch {epoch + 1} Validation", leave=False)

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(progress_bar):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images).logits  # logits from ViTForImageClassification
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Update progress bar with batch loss and accuracy
            progress_bar.set_postfix({'Batch Loss': loss.item(), 'Batch Acc': 100 * correct / total})

    # Return average loss and accuracy for the epoch
    return running_loss / total, correct / total

# Cross-validation function for hyperparameter tuning
def cross_validate_hyperparameter_tuning(X, y, hyperparameters, device, total_folds=5):
    outer_fold = stratifiedKFold(n_split=total_folds, shuffle=True, random_state=42)  # 5-fold CV for hyperparameter tuning
    outer_results = []

   # Get the product of all hyperparameter combinations
    all_combinations = list(itertools.product(*hyperparameters.values()))
    # Total number of combinations
    total_combinations = len(all_combinations)

    for fold, (train_idx, test_idx) in enumerate(outer_fold.split(X, y), 1):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Set up hyperparameter combinations
        param_keys = [list(d.keys())[0] for d in hyperparameters]
        param_values = [list(d.values())[0] for d in hyperparameters]
        combinations = product(*param_values)
        hyper_combo = [{param_keys[i]: combo[i] for i in range(len(param_keys))} for combo in combinations]

        print(f"\n=== Fold {fold}/{total_folds} ===")

        for combo_num, param in enumerate(hyper_combo, 1):
            LEARNING_RATE = param['lr']
            ATTENTION_HEADS = param['attention_heads']

            # Initialize the model with current hyperparameters
            model = ViTForImageClassification.from_pretrained(MODEL_NAME, num_labels=len(label_encoder.classes_), num_attention_heads=ATTENTION_HEADS)
            model.to(device)
            optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
            criterion = nn.CrossEntropyLoss()

            best_val_loss = np.inf
            epochs_no_improve = 0  # Early stopping counter

            print(f"Starting hyperparameter combination {combo_num}/{total_combinations}: lr={LEARNING_RATE}, attention_heads={ATTENTION_HEADS}")

            # Train on inner folds with early stopping
            for epoch in range(EPOCHS):
                # Create datasets
                train_dataset = SkinCancerDataset(X_train, IMAGE_DIRS, feature_extractor, transform=train_transform)
                val_dataset = SkinCancerDataset(X_test, IMAGE_DIRS, feature_extractor, transform=val_test_transform)

                train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
                val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

                train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, device, epoch)
                val_loss, val_acc = eval_epoch(model, val_loader, criterion, device, epoch)

                # Print progress for each epoch
                print(f'Epoch {epoch + 1}/{EPOCHS} (Fold {fold}, Combo {combo_num}): Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%, '
                      f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%')

                # Check if validation loss improved
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    epochs_no_improve = 0  # Reset counter if validation improves
                else:
                    epochs_no_improve += 1

                # Early stopping if no improvement for PATIENCE epochs
                if epochs_no_improve == PATIENCE:
                    print(f"Early stopping at epoch {epoch + 1} (Fold {fold}, Combo {combo_num})")
                    break

            # Store results for the fold and combination
            outer_results.append((param, val_acc))

            # Print progress for hyperparameter tuning
            print(f"Completed {combo_num}/{total_combinations} hyperparameter combinations for Fold {fold}")

    return outer_results

# Hyperparameters to test (learning rate and attention heads)
hyperparameters = [
    {'lr': [1e-3, 1e-4]},
    {'attention_heads': [8, 16]}  # Example values
]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [1]:


# Running hyperparameter tuning with progress tracking
results_tuning = cross_validate_hyperparameter_tuning(metadata, metadata['dx_encoded'], hyperparameters, device)

# Output the best hyperparameters and accuracy
best_result = max(results_tuning, key=lambda x: x[1])
print("Best Hyperparameters:", best_result[0])
print("Best Accuracy:", best_result[1])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(



=== Fold 1/5 ===


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch32-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting hyperparameter combination 1/2: lr=0.001, attention_heads=8


Epoch 1/10 (Fold 1, Combo 1): Train Loss: 1.0969, Train Acc: 64.84%, Val Loss: 0.9243, Val Acc: 65.94%


...

Epoch 6/10 (Fold 5, Combo 4): Train Loss: 0.3704, Train Acc: 88.40%, Val Loss: 0.4812, Val Acc: 83.85%


Epoch 7/10 (Fold 5, Combo 4): Train Loss: 0.2980, Train Acc: 90.99%, Val Loss: 0.4650, Val Acc: 84.75%


Epoch 8/10 (Fold 5, Combo 4): Train Loss: 0.2712, Train Acc: 91.50%, Val Loss: 0.4766, Val Acc: 84.70%


Epoch 9/10 (Fold 5, Combo 4): Train Loss: 0.1997, Train Acc: 94.24%, Val Loss: 0.4213, Val Acc: 86.25%


Epoch 10/10 (Fold 5, Combo 4): Train Loss: 0.1618, Train Acc: 95.11%, Val Loss: 0.4682, Val Acc: 84.25%
Completed 4/2 hyperparameter combinations for Fold 5
Best Hyperparameters: {'lr': 0.0001, 'attention_heads': 16}
Best Accuracy: 0.8618453865336658


In [3]:
# Loop through all the hyperparameter test results
for i, (params, accuracy) in enumerate(results_tuning):
    print(f"Test {i+1}: Hyperparameters: {params}, Accuracy: {accuracy:.4f}")

# Output the best hyperparameters and accuracy
best_result = max(results_tuning, key=lambda x: x[1])
print("\nBest Hyperparameters:", best_result[0])
print("Best Accuracy:", best_result[1])

Test 1: Hyperparameters: {'lr': 0.001, 'attention_heads': 8}, Accuracy: 0.7397
Test 2: Hyperparameters: {'lr': 0.001, 'attention_heads': 16}, Accuracy: 0.7651
Test 3: Hyperparameters: {'lr': 0.0001, 'attention_heads': 8}, Accuracy: 0.8244
Test 4: Hyperparameters: {'lr': 0.0001, 'attention_heads': 16}, Accuracy: 0.8544
Test 5: Hyperparameters: {'lr': 0.001, 'attention_heads': 8}, Accuracy: 0.7162
Test 6: Hyperparameters: {'lr': 0.001, 'attention_heads': 16}, Accuracy: 0.7516
Test 7: Hyperparameters: {'lr': 0.0001, 'attention_heads': 8}, Accuracy: 0.8369
Test 8: Hyperparameters: {'lr': 0.0001, 'attention_heads': 16}, Accuracy: 0.8618
Test 9: Hyperparameters: {'lr': 0.001, 'attention_heads': 8}, Accuracy: 0.7294
Test 10: Hyperparameters: {'lr': 0.001, 'attention_heads': 16}, Accuracy: 0.7369
Test 11: Hyperparameters: {'lr': 0.0001, 'attention_heads': 8}, Accuracy: 0.8263
Test 12: Hyperparameters: {'lr': 0.0001, 'attention_heads': 16}, Accuracy: 0.8587
Test 13: Hyperparameters: {'lr': 0.00

In [ ]:
# Constants for saving models
RESULT_SAVE_DIR = 'drive/MyDrive/ViT_Result'  # Directory to save results
MODEL_SAVE_DIR = 'drive/MyDrive/ViT_Model'  # Directory to save models

best_params = {'lr': 0.0001, 'attention_heads': 16}

# Configurable hyperparameters
EPOCHS = 100   # Set the number of epochs here
BATCH_SIZE = 512  # Set the batch size here
PATIENCE = 10 # Early stopping patience (number of epochs with no improvement)

# Create a training function for each fold
def train_model(X_train, y_train, X_test, y_test, device, params, fold):
    model = ViTForImageClassification.from_pretrained(
        MODEL_NAME,
        num_labels=len(set(y_train)),
        num_attention_heads=params['attention_heads']
    )
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=params['lr'])
    criterion = nn.CrossEntropyLoss()

    train_dataset = SkinCancerDataset(X_train, IMAGE_DIRS, feature_extractor, transform=train_transform)
    test_dataset = SkinCancerDataset(X_test, IMAGE_DIRS, feature_extractor, transform=val_test_transform)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    best_val_acc = 0
    patience_counter = 0

    # To store the epoch-wise results
    epoch_results = {
        'epoch': [],
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }

    # Training loop
    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_acc = correct / total

        # Validation step
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images).logits
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = correct / total

        # Append the results for the current epoch
        epoch_results['epoch'].append(epoch + 1)
        epoch_results['train_loss'].append(train_loss / total)
        epoch_results['train_acc'].append(train_acc * 100)
        epoch_results['val_loss'].append(val_loss / total)
        epoch_results['val_acc'].append(val_acc * 100)

        # Print epoch progress
        print(f"Fold {fold+1}, Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%")

        # Early stopping logic
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= PATIENCE:
            print("Early stopping triggered.")
            break

    # Save epoch-wise results to CSV file
    epoch_df = pd.DataFrame(epoch_results)
    accuracy_file_name = f"{RESULT_SAVE_DIR}/fold{fold+1}_accuracy.csv"
    epoch_df.to_csv(accuracy_file_name, index=False)
    print(f"Epoch accuracy and loss saved to {accuracy_file_name}")

    # After training, evaluate on the test fold and save predictions
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images).logits
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            actuals.extend(labels.cpu().numpy())

    # Save results to a CSV file
    result_df = pd.DataFrame({
        'Prediction': predictions,
        'Actual': actuals
    })
    file_name = f"{RESULT_SAVE_DIR}/fold{fold+1}.csv"
    result_df.to_csv(file_name, index=False)
    print(f"Fold {fold+1} predictions saved to {file_name}")

    return model, best_val_acc

# Main 20-fold Cross-Validation with stratifiedKFold
X = metadata
y = metadata['dx_encoded']

# Initialize stratified KFold with 20 splits
skf = stratifiedKFold(n_split=20, shuffle=True, random_state=42)

best_model_state_dict = None  # To store the best model's state_dict
best_model_acc = 0  # To track the best validation accuracy across folds

# Iterate through stratified KFold splits
for fold_idx, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    print(f"\n=== Starting Fold {fold_idx+1}/20 ===")
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Train the model and test
    model, val_acc = train_model(X_train, y_train, X_test, y_test, device, best_params, fold_idx)

    # Check if this fold's model has the best validation accuracy so far
    if val_acc > best_model_acc:
        best_model_acc = val_acc
        best_model_state_dict = model.state_dict()  # Save the best model state
        # Save the best model's state_dict to a file
        best_model_path = os.path.join(MODEL_SAVE_DIR, 'best_model.pth')
        torch.save(best_model_state_dict, best_model_path)
        print(f"New best model saved from Fold {fold_idx+1} with validation accuracy: {val_acc:.4f}")



=== Starting Fold 1/20 ===


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch32-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fold 1, Epoch 1/100 - Train Loss: 11259.0754, Train Acc: 63.81%, Val Loss: 487.1171, Val Acc: 67.66%
Fold 1, Epoch 2/100 - Train Loss: 8187.9549, Train Acc: 71.05%, Val Loss: 415.6670, Val Acc: 74.01%
Fold 1, Epoch 3/100 - Train Loss: 6885.5593, Train Acc: 76.82%, Val Loss: 344.3634, Val Acc: 79.37%
Fold 1, Epoch 4/100 - Train Loss: 5721.7124, Train Acc: 81.04%, Val Loss: 321.1661, Val Acc: 78.17%
Fold 1, Epoch 5/100 - Train Loss: 4969.4738, Train Acc: 83.61%, Val Loss: 278.1323, Val Acc: 81.15%
Fold 1, Epoch 6/100 - Train Loss: 4538.0464, Train Acc: 84.78%, Val Loss: 261.3093, Val Acc: 83.13%
Fold 1, Epoch 7/100 - Train Loss: 3734.1205, Train Acc: 88.21%, Val Loss: 265.8607, Val Acc: 81.94%
Fold 1, Epoch 8/100 - Train Loss: 3252.6198, Train Acc: 89.69%, Val Loss: 273.0413, Val Acc: 81.15%
Fold 1, Epoch 9/100 - Train Loss: 2852.2734, Train Acc: 91.05%, Val Loss: 236.3373, Val Acc: 83.53%
Fold 1, Epoch 10/100 - Train Loss: 2336.1528, Train Acc: 92.99%, Val Loss: 272.0247, Val Acc: 83.13

In [ ]:
# Precision-recall curves